In [ ]:
# Set the Datalake Access Key configuration
spark.conf.set(
    "fs.azure.account.key.cryptoanalyticslake.dfs.core.windows.net",
    dbutils.secrets.get(scope="key-vault-secret-scope",key="cryptoanalyticslake-access-key"))

In [ ]:
# Set Day Month Year
from datetime import datetime

today = datetime.utcnow()
year = today.year
month = today.month
day = today.day

In [ ]:
# Recursive data load for all files from a day from every partition in the Event Hub Namespace
sourcefolderpath = f"abfss://crypto-bronze@cryptoanalyticslake.dfs.core.windows.net/quotes-by-day/{year}/{month:0>2d}/{day:0>2d}"

print(sourcefolderpath)

df = spark.read.option("recursiveFileLookup","true").parquet(sourcefolderpath)

In [ ]:
display(df)

In [ ]:
# Define the input and output formats and paths and the table name.
write_format = 'delta'
save_path = '/tmp/delta/unmanaged/crypt-quotes'
table_name = 'default.unmanagedcryptoquotes'

In [ ]:
# Write the data to its target.
df.write \
  .format(write_format) \
  .save(save_path)

# Create the table.
spark.sql("CREATE TABLE " + table_name + " USING DELTA LOCATION '" + save_path + "'")

In [ ]:
delta_table_name = "unmanagedcryptoquotes"

display(spark.sql("SELECT * FROM " + delta_table_name))